# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
# with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
with open(os.path.join('.', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 

print(f"The number of twits in dataset is {len(twits['data'])}")

The number of twits in dataset is 1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

In [5]:
re.sub(r'[^a-zA-Z]+',' ', '26.00...ill')

' ill'

In [6]:
re.findall(r'[^a-zA-Z]', '26.00...ill')

['2', '6', '.', '0', '0', '.', '.', '.']

### Pre-Processing

In [28]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'http[^\s]+',' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$\S+',' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'@[^\s]+',' ', text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-zA-Z]+',' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(word, pos='v') for word in tokens if len(word)>1]
    
    return tokens

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mohammed\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
# my_text = r"RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG "
# my_symbol_pattern = re.compile(r'\s*\$\S+\s*')
# my_symbol_matches = re.finditer(my_symbol_pattern, my_text)
# for i,match in enumerate(my_symbol_matches):
#     print(i, match)
# print(re.sub(r'\$\S+',' ', my_text).split())

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [30]:
tokenized = [preprocess(message) for message in messages]

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [31]:
for message in tokenized[:5]:
    for word in message:
        print(word)

great
buy
at
ill
wait
staanalystalert
for
jefferies
maintain
with
rat
of
hold
set
target
price
at
usd
our
own
verdict
be
buy
hear
there
guy
who
know
someone
who
think
somebody
know
something
on
stocktwits
reveal
yourself


In [32]:
from collections import Counter, defaultdict


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

bow = defaultdict(int)
for message in tokenized:
    for word in message:
        bow[word] += 1
# bow = Counter([i for sub in tokenized for i in sub])


In [33]:
bow['the'],bow['it'],bow['and'],bow['wait'], bow['stocktwits'], bow['msft']


(398753, 167734, 208471, 19007, 1439, 1336)

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [76]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = bow

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 6

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 138450#120000

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word for word in freqs if freqs[word]>high_cutoff]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print('most common: ',K_most_common)
print(len(filtered_words) )
print('filtered: ',filtered_words[:10])

most common:  ['at', 'for', 'of', 'be', 'on', 'the', 'to', 'in', 'and', 'this', 'it']
21026
filtered:  ['great', 'buy', 'ill', 'wait', 'staanalystalert', 'jefferies', 'maintain', 'with', 'rat', 'hold']


In [66]:
# import pandas as pd
# df_freq = pd.DataFrame(list(freqs.values()), index=list(freqs.keys()), columns=['freq'])
# df_freq.head()

freq    68855
Name: with, dtype: int64

In [77]:
# # df_freq.query('freq > 5').sort_values(by='freq', ascending=False).tail(50)
# df_freq.sort_values(by='freq', ascending=False).head(10)

,freq
be,581222
the,398753
to,379487
for,273537
on,241663
of,211334
and,208471
in,205263
this,203540
it,167734


### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [78]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word:i+1 for i,word in enumerate(filtered_words)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {v:k for k,v in vocab.items()}
# tokenized with the words not in `filtered_words` removed.
filtered = [[word for word in sub if word in filtered_words] for sub in tokenized]

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [79]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [80]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1942282078707846

Finally let's convert our tokens into integer ids which we can pass to the network.

In [81]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

In [82]:
token_ids[:4]

[[1, 2, 3, 4],
 [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 2],
 [18, 19, 20, 21, 22, 23, 21, 24, 25, 22, 26, 27],
 [30, 31, 32, 33, 34, 35, 36, 37]]

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [83]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout_prob = dropout
        
        print('INFO: ',vocab_size, embed_size, lstm_size, output_size, lstm_layers, dropout)
        
        # TODO Implement
        
        # Setup embedding layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_size).cuda()
        self.lstm = nn.LSTM(self.embed_size, self.lstm_size, self.lstm_layers,
                           dropout=self.dropout_prob, batch_first=False).cuda()
        # Setup additional layers
        self.dropout = nn.Dropout(self.dropout_prob).cuda()
        
        self.fc = nn.Linear(self.lstm_size, self.output_size).cuda()
        self.softmax = nn.LogSoftmax(dim=-1).cuda()
        


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
#         train_on_gpu = torch.cuda.is_available()
        if (training_on_gpu):
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
                     weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
        else:
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                     weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        _batch_size = nn_input.size(1)
        
        embeds = self.embedding(nn_input).cuda()
        lstm_out, hidden = self.lstm(embeds, hidden_state)
        lstm_out = lstm_out[-1, :,:]
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        soft_out = self.softmax(out)
        
        
        return soft_out, hidden

### View Model

In [84]:
training_on_gpu = True
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64).cuda()
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

INFO:  21026 10 6 5 2 0.1
tensor([[-1.8267, -1.5151, -1.1546, -1.8474, -1.9210],
        [-1.7457, -1.4424, -1.2426, -1.8756, -1.9159],
        [-1.7990, -1.4549, -1.1764, -1.9259, -1.9175],
        [-1.7268, -1.4463, -1.2580, -1.8620, -1.9164]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [85]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [86]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# # TODO Implement 

# train_features = pass
# valid_features = pass
# train_labels = pass
# valid_labels = pass

from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
train_features, valid_features, train_labels, valid_labels = train_test_split(token_ids, 
                                                    sentiments, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

In [87]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
print(type(text_batch))
logps, hidden = model.forward(text_batch.cuda(), hidden)
print(logps.size())

INFO:  21027 200 128 5 1 0.0
<class 'torch.Tensor'>
torch.Size([64, 5])


### Training
It's time to train the neural network!

In [88]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

INFO:  21027 1024 512 5 2 0.2


TextClassifier(
  (embedding): Embedding(21027, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (softmax): LogSoftmax()
)

In [90]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 5
batch_size = 512
learning_rate = 0.0005

running_loss = 0

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

train_labels = train_labels[:batch_size*int(len(train_labels)/batch_size)]
train_features = train_features[:batch_size*int(len(train_features)/batch_size)]
valid_labels = valid_labels[:batch_size*int(len(valid_labels)/batch_size)]
valid_features = valid_features[:batch_size*int(len(valid_features)/batch_size)]


for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        hidden = model.init_hidden(batch_size)
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        optimizer.zero_grad()
        output, hidden = model.forward(text_batch, hidden)
        loss = criterion(output, labels.long())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            val_h = model.init_hidden(batch_size)
            model.eval()
            
            # TODO Implement: Print metrics
            with torch.no_grad():
                for inputs, labels in dataloader(
            valid_features, valid_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
                    inputs, labels = inputs.to(device), labels.to(device)
                    val_h = tuple([each.data for each in val_h])
                    
                    output, val_h = model.forward(inputs, val_h)
                    values, indices = torch.max(output, 1)
                    
                    batch_loss = criterion(output, labels.long())
                    
                    test_loss += batch_loss.item()   
                    
                    ps = torch.exp(output)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(valid_labels):.5f}.. "
                  f"Test accuracy: {accuracy/batch_size:.3f}")
            running_loss = 0
            model.train()

Starting epoch 1
Epoch 1/5.. Train loss: 0.876.. Test loss: 0.00185.. Test accuracy: 0.491
Epoch 1/5.. Train loss: 0.826.. Test loss: 0.00179.. Test accuracy: 0.500
Epoch 1/5.. Train loss: 0.811.. Test loss: 0.00174.. Test accuracy: 0.504
Epoch 1/5.. Train loss: 0.787.. Test loss: 0.00184.. Test accuracy: 0.488
Epoch 1/5.. Train loss: 0.780.. Test loss: 0.00177.. Test accuracy: 0.497
Epoch 1/5.. Train loss: 0.768.. Test loss: 0.00228.. Test accuracy: 0.426
Epoch 1/5.. Train loss: 0.766.. Test loss: 0.00188.. Test accuracy: 0.478
Epoch 1/5.. Train loss: 0.752.. Test loss: 0.00186.. Test accuracy: 0.484
Epoch 1/5.. Train loss: 0.749.. Test loss: 0.00177.. Test accuracy: 0.495
Epoch 1/5.. Train loss: 0.738.. Test loss: 0.00194.. Test accuracy: 0.476
Epoch 1/5.. Train loss: 0.727.. Test loss: 0.00182.. Test accuracy: 0.493
Epoch 1/5.. Train loss: 0.730.. Test loss: 0.00192.. Test accuracy: 0.473
Epoch 1/5.. Train loss: 0.728.. Test loss: 0.00195.. Test accuracy: 0.469
Epoch 1/5.. Train los

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [91]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in filtered_words]
    # Convert words to ids
    tokens = [vocab[word] for word in tokens]
        
    # Adding a batch dimension
#     count = 1
#     for text_input, labels in dataloader([tokens], [0], sequence_length=20, batch_size=5, shuffle=False):
#         print('batch: ', count)
#         count += 1
#         print(text_input)
#         text_input = text_input
#     text_input = text_input.cuda()
#     # Get the NN output
#     hidden = model.init_hidden(5)
#     for each in hidden:
#         each.to('cuda')
#     logps, _ = model.forward(text_input, hidden)
    tokens = torch.tensor(tokens).cuda()
    tokens = tokens.view(-1, 1)
    hidden = model.init_hidden(1)
    for each in hidden:
        each.to('cuda')
    logps, _ = model.forward(tokens, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [92]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cuda")
predict(text, model, vocab)

tensor([[1.2033e-04, 3.8170e-02, 1.7041e-02, 7.3291e-01, 2.1176e-01]],
       device='cuda:0', grad_fn=<ExpBackward>)

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** The model predicts that the sentiment score is (4/5) with a very high degree of certainty (all values are practically zero expect for the predicted score and 5/5 (which is 0.21). In other words, the prediction is positive with a chance of (0.733 + 0.212 = 0.945)**

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [93]:
with open(os.path.join('.', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [94]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [95]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [96]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[0.2232, 0.1400, 0.1038, 0.0967, 0.4363]], device='cuda:0',
        grad_fn=<ExpBackward>),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.